In [25]:
import PyPDF2
import camelot
import pandas as pd

In [26]:
#FAILED PYPDF IMPLEMENTATION

# text = ""
# with open("engg-credit-transfer-database.pdf", 'rb') as file:
#     reader = PyPDF2.PdfReader(file)
#     for page in reader.pages:
#         text += page.extract_text()

# with open("tester.txt", "w+") as tester_file:
#     tester_file.write(text)

In [27]:
bbms_file = open("bbms-credit-transfer-database.pdf", 'rb')

#store data in pdfReader
pdfReader = PyPDF2.PdfReader(bbms_file)

#count number of pages
totalPages = len(pdfReader.pages)
totalPages

20

In [28]:
bbms_pdf = camelot.read_pdf("bbms-credit-transfer-database.pdf", 
                       pages='1-end') #address of file location
initial_df = pd.DataFrame()

#add data to DF
for i in range(totalPages):
    if i == 0: initial_df = bbms_pdf[i].df
    else: initial_df = pd.concat([initial_df, bbms_pdf[i].df], ignore_index=True)

In [29]:
#first row is header
initial_df.columns = initial_df.iloc[0]  
initial_df = initial_df[1:]              
initial_df

,Exchange/\nArticulation,Exchange Institution,Country,Course (Exchange Institution),Credit Units \n(Exchange \nInstitution),Transferred Course (HKU)
1,exchange,"Murdoch University, Australia",Australia,BMS324 Clinical Microbiology I,3,BBMS3004\nMolecular Diagnostics Laboratory
2,exchange,"Murdoch University, Australia",Australia,BMS327 Diagnostic Genome,3,BBMS3009\nGenome Science
3,exchange,"Murdoch University, Australia",Australia,"EGL101 Drama, Theatre and Performances",3,Elective
4,exchange,"Murdoch University, Australia",Australia,PHO124 Photography 1,3,Elective
5,exchange,"the University of Leeds, United Kingdom",Australia,BMSC1212\nIntroduction to Pharmacology,10,BBMS2005\nBiomedical Pharmacology
...,...,...,...,...,...,...
423,Exchange,"University of California at Berkeley, USA",USA,"GWS14001 Gender, Sexuality & Race in Global \n...",4,GEND2XXX\nAdvanced elective
424,Exchange,"Northeastern University, USA",USA,BIOL4707\nCell and Molecular biology,4,BBMS3002\nMolecular Biology of the Cell
425,Exchange,"Northeastern University, USA",USA,BIOL3421 & BIOL3422\nMicrobiology & Lab for Mi...,5,BBMS3001\nMedical Microbiology
426,Exchange,"Northeastern University, USA",USA,PSYC3466\nCognition,4,PSYC2007\nCognitive psychology


In [30]:
#ucb weird course  naming convention
initial_df['Course (Exchange Institution)'] = initial_df['Course (Exchange Institution)'].str.replace('PB HLTH', 'PBHLTH')
initial_df['Exchange-Course-Code'] = ''

In [31]:
#course code from course info extraction
def populate_exchange_course_code(course):
    words = course.split()
    
    if len(words) > 1 and any(char.isdigit() for char in words[1]):
        return ' '.join(words[:2])  

    exchange_code_parts = []
    for word in words:
        if any(char.isdigit() for char in word) or '-' in word or '/' in word or '&' in word or '.' in word:
            exchange_code_parts.append(word)
        else:
            break
    return ' '.join(exchange_code_parts)

initial_df['Exchange-Course-Code'] = initial_df['Course (Exchange Institution)'].apply(populate_exchange_course_code)
initial_df

,Exchange/\nArticulation,Exchange Institution,Country,Course (Exchange Institution),Credit Units \n(Exchange \nInstitution),Transferred Course (HKU),Exchange-Course-Code
1,exchange,"Murdoch University, Australia",Australia,BMS324 Clinical Microbiology I,3,BBMS3004\nMolecular Diagnostics Laboratory,BMS324
2,exchange,"Murdoch University, Australia",Australia,BMS327 Diagnostic Genome,3,BBMS3009\nGenome Science,BMS327
3,exchange,"Murdoch University, Australia",Australia,"EGL101 Drama, Theatre and Performances",3,Elective,EGL101
4,exchange,"Murdoch University, Australia",Australia,PHO124 Photography 1,3,Elective,PHO124
5,exchange,"the University of Leeds, United Kingdom",Australia,BMSC1212\nIntroduction to Pharmacology,10,BBMS2005\nBiomedical Pharmacology,BMSC1212
...,...,...,...,...,...,...,...
423,Exchange,"University of California at Berkeley, USA",USA,"GWS14001 Gender, Sexuality & Race in Global \n...",4,GEND2XXX\nAdvanced elective,GWS14001
424,Exchange,"Northeastern University, USA",USA,BIOL4707\nCell and Molecular biology,4,BBMS3002\nMolecular Biology of the Cell,BIOL4707
425,Exchange,"Northeastern University, USA",USA,BIOL3421 & BIOL3422\nMicrobiology & Lab for Mi...,5,BBMS3001\nMedical Microbiology,BIOL3421 & BIOL3422
426,Exchange,"Northeastern University, USA",USA,PSYC3466\nCognition,4,PSYC2007\nCognitive psychology,PSYC3466


In [32]:
#remove from initial course info
def remove_exchange_code(course, exchange_code):
    return course.replace(exchange_code, '').strip()

initial_df['Course (Exchange Institution)'] = initial_df.apply(lambda row: remove_exchange_code(row['Course (Exchange Institution)'], row['Exchange-Course-Code']), axis=1)

initial_df

,Exchange/\nArticulation,Exchange Institution,Country,Course (Exchange Institution),Credit Units \n(Exchange \nInstitution),Transferred Course (HKU),Exchange-Course-Code
1,exchange,"Murdoch University, Australia",Australia,Clinical Microbiology I,3,BBMS3004\nMolecular Diagnostics Laboratory,BMS324
2,exchange,"Murdoch University, Australia",Australia,Diagnostic Genome,3,BBMS3009\nGenome Science,BMS327
3,exchange,"Murdoch University, Australia",Australia,"Drama, Theatre and Performances",3,Elective,EGL101
4,exchange,"Murdoch University, Australia",Australia,Photography 1,3,Elective,PHO124
5,exchange,"the University of Leeds, United Kingdom",Australia,Introduction to Pharmacology,10,BBMS2005\nBiomedical Pharmacology,BMSC1212
...,...,...,...,...,...,...,...
423,Exchange,"University of California at Berkeley, USA",USA,"Gender, Sexuality & Race in Global \nPolitical...",4,GEND2XXX\nAdvanced elective,GWS14001
424,Exchange,"Northeastern University, USA",USA,Cell and Molecular biology,4,BBMS3002\nMolecular Biology of the Cell,BIOL4707
425,Exchange,"Northeastern University, USA",USA,Microbiology & Lab for Microbiology,5,BBMS3001\nMedical Microbiology,BIOL3421 & BIOL3422
426,Exchange,"Northeastern University, USA",USA,Cognition,4,PSYC2007\nCognitive psychology,PSYC3466


In [33]:
# remove country from university name
def remove_after_last_comma(course):
    parts = course.rsplit(',', 1)  
    return parts[0] if len(parts) > 1 else course  

initial_df['Exchange Institution'] = initial_df['Exchange Institution'].apply(remove_after_last_comma)
initial_df

,Exchange/\nArticulation,Exchange Institution,Country,Course (Exchange Institution),Credit Units \n(Exchange \nInstitution),Transferred Course (HKU),Exchange-Course-Code
1,exchange,Murdoch University,Australia,Clinical Microbiology I,3,BBMS3004\nMolecular Diagnostics Laboratory,BMS324
2,exchange,Murdoch University,Australia,Diagnostic Genome,3,BBMS3009\nGenome Science,BMS327
3,exchange,Murdoch University,Australia,"Drama, Theatre and Performances",3,Elective,EGL101
4,exchange,Murdoch University,Australia,Photography 1,3,Elective,PHO124
5,exchange,the University of Leeds,Australia,Introduction to Pharmacology,10,BBMS2005\nBiomedical Pharmacology,BMSC1212
...,...,...,...,...,...,...,...
423,Exchange,University of California at Berkeley,USA,"Gender, Sexuality & Race in Global \nPolitical...",4,GEND2XXX\nAdvanced elective,GWS14001
424,Exchange,Northeastern University,USA,Cell and Molecular biology,4,BBMS3002\nMolecular Biology of the Cell,BIOL4707
425,Exchange,Northeastern University,USA,Microbiology & Lab for Microbiology,5,BBMS3001\nMedical Microbiology,BIOL3421 & BIOL3422
426,Exchange,Northeastern University,USA,Cognition,4,PSYC2007\nCognitive psychology,PSYC3466


In [44]:
#remove unnecessary line breaks
initial_df = initial_df.map(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
initial_df

,Exchange/\nArticulation,Exchange Institution,Country,Course (Exchange Institution),Credit Units \n(Exchange \nInstitution),Transferred Course (HKU),Exchange-Course-Code,HKU-Course-Code
1,exchange,Murdoch University,Australia,Clinical Microbiology I,3,Molecular Diagnostics Laboratory,BMS324,BBMS3004
2,exchange,Murdoch University,Australia,Diagnostic Genome,3,Genome Science,BMS327,BBMS3009
3,exchange,Murdoch University,Australia,"Drama, Theatre and Performances",3,Elective,EGL101,
4,exchange,Murdoch University,Australia,Photography 1,3,Elective,PHO124,
5,exchange,the University of Leeds,Australia,Introduction to Pharmacology,10,Biomedical Pharmacology,BMSC1212,BBMS2005
...,...,...,...,...,...,...,...,...
423,Exchange,University of California at Berkeley,USA,"Gender, Sexuality & Race in Global Political ...",4,Advanced elective,GWS14001,GEND2XXX
424,Exchange,Northeastern University,USA,Cell and Molecular biology,4,Molecular Biology of the Cell,BIOL4707,BBMS3002
425,Exchange,Northeastern University,USA,Microbiology & Lab for Microbiology,5,Medical Microbiology,BIOL3421 & BIOL3422,BBMS3001
426,Exchange,Northeastern University,USA,Cognition,4,Cognitive psychology,PSYC3466,PSYC2007


In [36]:
# same but for hku
def get_hku_course_code(course):
    first_word = course.split()[0]  
    return first_word if any(char.isdigit() for char in first_word) else ''

initial_df['HKU-Course-Code'] = initial_df['Transferred Course (HKU)'].apply(get_hku_course_code)

initial_df['Transferred Course (HKU)'] = initial_df.apply(
    lambda row: row['Transferred Course (HKU)'].replace(row['HKU-Course-Code'], '').strip(),
    axis=1
)
initial_df

,Exchange/\nArticulation,Exchange Institution,Country,Course (Exchange Institution),Credit Units \n(Exchange \nInstitution),Transferred Course (HKU),Exchange-Course-Code,HKU-Course-Code
1,exchange,Murdoch University,Australia,Clinical Microbiology I,3,Molecular Diagnostics Laboratory,BMS324,BBMS3004
2,exchange,Murdoch University,Australia,Diagnostic Genome,3,Genome Science,BMS327,BBMS3009
3,exchange,Murdoch University,Australia,"Drama, Theatre and Performances",3,Elective,EGL101,
4,exchange,Murdoch University,Australia,Photography 1,3,Elective,PHO124,
5,exchange,the University of Leeds,Australia,Introduction to Pharmacology,10,Biomedical Pharmacology,BMSC1212,BBMS2005
...,...,...,...,...,...,...,...,...
423,Exchange,University of California at Berkeley,USA,"Gender, Sexuality & Race in Global Political ...",4,Advanced elective,GWS14001,GEND2XXX
424,Exchange,Northeastern University,USA,Cell and Molecular biology,4,Molecular Biology of the Cell,BIOL4707,BBMS3002
425,Exchange,Northeastern University,USA,Microbiology & Lab for Microbiology,5,Medical Microbiology,BIOL3421 & BIOL3422,BBMS3001
426,Exchange,Northeastern University,USA,Cognition,4,Cognitive psychology,PSYC3466,PSYC2007


In [42]:
# filter df, clean data
columns_to_keep = [
    'Exchange Institution',
    'Country',
    'Course (Exchange Institution)',
    'Transferred Course (HKU)',
    'Exchange-Course-Code',
    'HKU-Course-Code'
]

filtered_df = initial_df[columns_to_keep]
filtered_df = filtered_df.rename(columns={
    'Exchange Institution': 'University',
    'Country': 'University-Country',
    'Course (Exchange Institution)': 'Exchange-Course-Title',
    'Transferred Course (HKU)': 'HKU-Course-Title'
})

final_order = [
    'University',
    'University-Country',
    'Exchange-Course-Code',
    'Exchange-Course-Title',
    'HKU-Course-Code',
    'HKU-Course-Title'
]

filtered_df = filtered_df[final_order]

filtered_df['University'] = filtered_df['University'].str.replace(r'^The\s+', '', regex=True)
filtered_df['University'] = filtered_df['University'].str.replace(r'^the\s+', '', regex=True)
filtered_df.loc[filtered_df['University'] == 'KOC University', 'University'] = 'Koc University'

filtered_df


,University,University-Country,Exchange-Course-Code,Exchange-Course-Title,HKU-Course-Code,HKU-Course-Title
1,Murdoch University,Australia,BMS324,Clinical Microbiology I,BBMS3004,Molecular Diagnostics Laboratory
2,Murdoch University,Australia,BMS327,Diagnostic Genome,BBMS3009,Genome Science
3,Murdoch University,Australia,EGL101,"Drama, Theatre and Performances",,Elective
4,Murdoch University,Australia,PHO124,Photography 1,,Elective
5,University of Leeds,Australia,BMSC1212,Introduction to Pharmacology,BBMS2005,Biomedical Pharmacology
...,...,...,...,...,...,...
423,University of California at Berkeley,USA,GWS14001,"Gender, Sexuality & Race in Global Political ...",GEND2XXX,Advanced elective
424,Northeastern University,USA,BIOL4707,Cell and Molecular biology,BBMS3002,Molecular Biology of the Cell
425,Northeastern University,USA,BIOL3421 & BIOL3422,Microbiology & Lab for Microbiology,BBMS3001,Medical Microbiology
426,Northeastern University,USA,PSYC3466,Cognition,PSYC2007,Cognitive psychology


In [43]:
# convert to csv
# filtered_df.to_csv("bbms-credit_transfer_database.csv",index=False)